# Financial Ratio Analysis Notebook

Interactive analysis of company financial ratios and performance benchmarking.

## Overview
This notebook provides comprehensive financial ratio analysis for multiple companies across different industries, with comparison against industry benchmarks.

## Companies Analyzed
- TechCorp Inc (Technology)
- Global Retail Co (Retail)
- ManufacturePro (Manufacturing)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import json
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

## 1. Data Loading and Exploration

In [ ]:
# Load financial data
df = pd.read_csv('../data/financial_statements.csv')
print("📊 Financial Data Overview:")
print(f"Dataset shape: {df.shape}")
print(f"Companies: {df['company_name'].unique()}")
print(f"Periods: {df['period'].unique()}")

# Display basic statistics
print("\n📈 Basic Statistics:")
df.describe()

In [ ]:
# Load industry benchmarks
with open('../data/industry_benchmarks.json', 'r') as f:
    benchmarks = json.load(f)

print("🏭 Industry Benchmarks Loaded:")
for industry, metrics in benchmarks['industry_benchmarks'].items():
    print(f"\n{industry}:")
    for category, ratios in metrics.items():
        print(f"  {category}: {len(ratios)} ratios")

## 2. Financial Ratio Calculations

In [ ]:
# Import our ratio calculator
import sys
sys.path.append('../scripts')
from ratio_calculator import RatioCalculator

# Calculate ratios for each company
ratio_results = {}
calculator = RatioCalculator()

for company_id in df['company_id'].unique():
    company_data = df[df['company_id'] == company_id].iloc[-1]  # Latest quarter
    company_name = company_data['company_name']
    
    ratios = calculator.calculate_all_ratios(company_data.to_dict())
    ratio_results[company_id] = {
        'company_name': company_name,
        'industry': company_data['industry'],
        'ratios': ratios
    }
    
print("✅ Financial ratios calculated for all companies")

## 3. Profitability Analysis

In [ ]:
# Extract profitability ratios
profitability_data = []
for company_id, data in ratio_results.items():
    profitability = data['ratios']['profitability']
    profitability_data.append({
        'Company': data['company_name'],
        'Industry': data['industry'],
        'Gross Margin': profitability['gross_margin'],
        'Operating Margin': profitability['operating_margin'],
        'Net Margin': profitability['net_margin'],
        'ROA': profitability['roa'],
        'ROE': profitability['roe']
    })

profit_df = pd.DataFrame(profitability_data)
print("💰 Profitability Ratios:")
profit_df

In [ ]:
# Visualize profitability comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Gross Margin
sns.barplot(data=profit_df, x='Company', y='Gross Margin', ax=axes[0,0], hue='Industry', dodge=False)
axes[0,0].set_title('Gross Margin by Company')
axes[0,0].tick_params(axis='x', rotation=45)

# Net Margin
sns.barplot(data=profit_df, x='Company', y='Net Margin', ax=axes[0,1], hue='Industry', dodge=False)
axes[0,1].set_title('Net Profit Margin by Company')
axes[0,1].tick_params(axis='x', rotation=45)

# ROA
sns.barplot(data=profit_df, x='Company', y='ROA', ax=axes[1,0], hue='Industry', dodge=False)
axes[1,0].set_title('Return on Assets (ROA)')
axes[1,0].tick_params(axis='x', rotation=45)

# ROE
sns.barplot(data=profit_df, x='Company', y='ROE', ax=axes[1,1], hue='Industry', dodge=False)
axes[1,1].set_title('Return on Equity (ROE)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Liquidity and Solvency Analysis

In [ ]:
# Extract liquidity and leverage ratios
liquidity_data = []
for company_id, data in ratio_results.items():
    liquidity = data['ratios']['liquidity']
    leverage = data['ratios']['leverage']
    liquidity_data.append({
        'Company': data['company_name'],
        'Industry': data['industry'],
        'Current Ratio': liquidity['current_ratio'],
        'Quick Ratio': liquidity['quick_ratio'],
        'Cash Ratio': liquidity['cash_ratio'],
        'Debt to Equity': leverage['debt_to_equity'],
        'Interest Coverage': min(leverage['interest_coverage'], 20)  # Cap for visualization
    })

liquidity_df = pd.DataFrame(liquidity_data)
print("💧 Liquidity and Leverage Ratios:")
liquidity_df

In [ ]:
# Create interactive Plotly chart for liquidity analysis
fig = px.scatter(liquidity_df, 
                 x='Current Ratio', 
                 y='Debt to Equity',
                 size='Quick Ratio',
                 color='Industry',
                 hover_data=['Company', 'Interest Coverage'],
                 title='Liquidity vs Leverage Analysis',
                 size_max=30)

# Add benchmark lines
fig.add_hline(y=1.0, line_dash="dash", line_color="red", annotation_text="High Debt Risk")
fig.add_vline(x=1.5, line_dash="dash", line_color="green", annotation_text="Good Liquidity")

fig.show()

## 5. Industry Benchmark Comparison

In [ ]:
# Compare companies against their industry benchmarks
comparison_results = []

for company_id, data in ratio_results.items():
    company_name = data['company_name']
    industry = data['industry']
    industry_benchmarks = benchmarks['industry_benchmarks'][industry]
    
    for category, ratios in data['ratios'].items():
        if category in industry_benchmarks:
            for ratio_name, ratio_value in ratios.items():
                if ratio_name in industry_benchmarks[category]:
                    benchmark_value = industry_benchmarks[category][ratio_name]
                    difference = ratio_value - benchmark_value
                    
                    comparison_results.append({
                        'Company': company_name,
                        'Industry': industry,
                        'Category': category,
                        'Ratio': ratio_name,
                        'Company_Value': ratio_value,
                        'Benchmark': benchmark_value,
                        'Difference': difference,
                        'Status': 'Above' if difference > 0 else 'Below'
                    })

comparison_df = pd.DataFrame(comparison_results)
print("📊 Benchmark Comparison Results:")
comparison_df.head(10)

In [ ]:
# Create heatmap of performance vs benchmarks
pivot_data = comparison_df.pivot_table(
    index=['Company', 'Industry'], 
    columns='Ratio', 
    values='Difference', 
    aggfunc='first'
)

plt.figure(figsize=(12, 6))
sns.heatmap(pivot_data, 
            annot=True, 
            cmap='RdYlGn', 
            center=0,
            fmt='.3f',
            linewidths=0.5)
plt.title('Performance vs Industry Benchmarks (Green = Above, Red = Below)')
plt.tight_layout()
plt.show()

## 6. Financial Health Scoring

In [ ]:
# Calculate financial health scores
health_scores = []

for company_id, data in ratio_results.items():
    company_name = data['company_name']
    industry = data['industry']
    
    score = 0
    total_metrics = 0
    insights = []
    
    # Profitability assessment
    profitability = data['ratios']['profitability']
    if profitability['net_margin'] > 0.1:
        score += 1
        insights.append("Strong profitability")
    total_metrics += 1
    
    # Liquidity assessment
    liquidity = data['ratios']['liquidity']
    if liquidity['current_ratio'] > 1.5:
        score += 1
        insights.append("Good liquidity position")
    total_metrics += 1
    
    # Leverage assessment
    leverage = data['ratios']['leverage']
    if leverage['debt_to_equity'] < 1.0:
        score += 1
        insights.append("Conservative leverage")
    total_metrics += 1
    
    health_percentage = (score / total_metrics) * 100
    
    if health_percentage >= 80:
        rating = "Excellent"
    elif health_percentage >= 60:
        rating = "Good"
    elif health_percentage >= 40:
        rating = "Fair"
    else:
        rating = "Poor"
    
    health_scores.append({
        'Company': company_name,
        'Industry': industry,
        'Health_Score': health_percentage,
        'Rating': rating,
        'Insights': ', '.join(insights)
    })

health_df = pd.DataFrame(health_scores)
print("🏥 Financial Health Assessment:")
health_df

## 7. Investment Recommendations

In [ ]:
# Generate investment recommendations
recommendations = []

for company_id, data in ratio_results.items():
    company_name = data['company_name']
    ratios = data['ratios']
    
    # Analyze strengths and weaknesses
    strengths = []
    weaknesses = []
    
    # Profitability analysis
    if ratios['profitability']['net_margin'] > 0.15:
        strengths.append("High profitability")
    elif ratios['profitability']['net_margin'] < 0.05:
        weaknesses.append("Low profit margins")
    
    # Liquidity analysis
    if ratios['liquidity']['current_ratio'] > 2.0:
        strengths.append("Strong liquidity")
    elif ratios['liquidity']['current_ratio'] < 1.0:
        weaknesses.append("Potential liquidity concerns")
    
    # Leverage analysis
    if ratios['leverage']['debt_to_equity'] < 0.5:
        strengths.append("Low debt levels")
    elif ratios['leverage']['debt_to_equity'] > 1.5:
        weaknesses.append("High financial leverage")
    
    # Generate recommendation
    if len(strengths) >= 2 and len(weaknesses) <= 1:
        recommendation = "BUY"
        reasoning = "Strong financial position with multiple strengths"
    elif len(strengths) >= len(weaknesses):
        recommendation = "HOLD"
        reasoning = "Balanced financial profile with some areas for improvement"
    else:
        recommendation = "SELL"
        reasoning = "Multiple financial weaknesses requiring attention"
    
    recommendations.append({
        'Company': company_name,
        'Recommendation': recommendation,
        'Reasoning': reasoning,
        'Strengths': ', '.join(strengths) if strengths else 'None',
        'Weaknesses': ', '.join(weaknesses) if weaknesses else 'None'
    })

reco_df = pd.DataFrame(recommendations)
print("💡 Investment Recommendations:")
reco_df

## Summary

This analysis provides comprehensive financial ratio assessment across multiple companies and industries. Key findings:

- **Profitability**: Identifies companies with strong vs weak profit margins
- **Liquidity**: Assesses short-term financial health and cash position
- **Leverage**: Evaluates debt levels and financial risk
- **Benchmarking**: Compares performance against industry standards
- **Investment Readiness**: Provides data-driven investment recommendations

Use these insights for informed investment decisions and financial analysis.